In [ ]:
import tensorflow as tf
import os,cv2
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import open3d as o3d

In [ ]:
path = "EHF/"

data = []
for images in tqdm(os.listdir(path)):
    if images.endswith('.jpg'):
        image_path = path + images
        image = cv2.imread(image_path)
        image = cv2.resize(image,(256,256))
        data.append(image)
data = np.array(data)

In [ ]:
plt.imshow(data[0])
plt.axis('off')
plt.show()

In [ ]:
def load_ply_data(ply_pat):
    pcd = o3d.io.read_point_cloud(ply_path)
    vertices = np.asarray(pcd.points)
    min_coords = np.min(vertices, axis=0)
    max_coords = np.max(vertices, axis=0)
    normalized_vertices = (vertices - min_coords) / (max_coords - min_coords)
    image_array = np.zeros((256, 256, 3), dtype=np.uint8)  # Adjust the image size as needed
    for vertex in normalized_vertices:
        x, y, z = vertex
        image_x = int(x * (image_array.shape[1] - 1))
        image_y = int((1 - y) * (image_array.shape[0] - 1))  # Reverse the y-coordinate
        image_array[image_y, image_x, :] = 255 
    image = Image.fromarray(image_array)
    image.save("check.jpg")
    img = cv2.imread("check.jpg")
    return img

In [ ]:
meshes = []

for images in tqdm(os.listdir(path)):
    if images.endswith('.ply'):
        ply_path = path + images
        ply_data = load_ply_data(ply_path)
        meshes.append(ply_data)
meshes = np.array(meshes)

In [ ]:
plt.imshow(meshes[0])
plt.axis('off')
plt.show()

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((data, meshes))
dataset = dataset.batch(batch_size=4)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Layer

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, concatenate, LeakyReLU,Concatenate,Flatten,Dense,BatchNormalization,AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

In [ ]:
class InstanceNormalization(Layer):
    def __init__(self, **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.scale = self.add_weight(name='scale',
                                     shape=input_shape[-1:],
                                     initializer=tf.random_normal_initializer(1., 0.02),
                                     trainable=True)
        self.offset = self.add_weight(name='offset',
                                      shape=input_shape[-1:],
                                      initializer='zeros',
                                      trainable=True)
        super(InstanceNormalization, self).build(input_shape)

    def call(self, x):
        mean, var = tf.nn.moments(x, axes=[1, 2], keepdims=True)
        inv = tf.math.rsqrt(var + 1e-5)
        normalized = (x - mean) * inv
        return self.scale * normalized + self.offset

In [ ]:
def build_generator():
    inputs = Input(shape=(256, 256,3))
    
    xi_and_y_i = inputs
    xi_yi_sz64 = AveragePooling2D(pool_size=2)(xi_and_y_i)
    xi_yi_sz32 = AveragePooling2D(pool_size=4)(xi_and_y_i)
    xi_yi_sz16 = AveragePooling2D(pool_size=8)(xi_and_y_i)
    xi_yi_sz8 = AveragePooling2D(pool_size=16)(xi_and_y_i)
    layer1 = Conv2D(64, kernel_size=4, strides=2, 
                   padding="same", name = 'layer1') (inputs)
    layer1 = LeakyReLU(alpha=0.2)(layer1)
    layer1 = concatenate([layer1, xi_yi_sz64]) # ==========
    layer2 = Conv2D(128, kernel_size=4, strides=2, 
                   padding="same", name = 'layer2') (layer1)
    layer2 = InstanceNormalization()(layer2, training=1)
    layer3 = LeakyReLU(alpha=0.2)(layer2)
    layer3 = concatenate([layer3, xi_yi_sz32]) # ==========
    layer3 = Conv2D(256, kernel_size=4, strides=2, 
                   padding="same", name = 'layer3') (layer3)
    
    layer3 = InstanceNormalization()(layer3, training=1)
    layer4 = LeakyReLU(alpha=0.2)(layer3)
    layer4 = concatenate([layer4, xi_yi_sz16]) # ==========
    layer4 = Conv2D(512, kernel_size=4, strides=2, 
                   padding="same", name = 'layer4') (layer4)
    
    layer4 = InstanceNormalization()(layer4, training=1)
    layer4 = LeakyReLU(alpha=0.2)(layer4)
    layer4 = concatenate([layer4, xi_yi_sz8]) # ==========
    
    layer9 = Conv2DTranspose(256, kernel_size=4, strides=2, 
                            kernel_initializer = tf.keras.initializers.GlorotNormal(seed=None), name = 'layer9')(layer4) 
    layer9 = Cropping2D(((1,1),(1,1)))(layer9)
    layer9 = InstanceNormalization()(layer9, training=1)
    layer9 = Concatenate(axis=-1)([layer9, layer3])
    layer9 = Activation('relu')(layer9)
    layer9 = concatenate([layer9, xi_yi_sz16]) # ==========
    layer10 = Conv2DTranspose(128, kernel_size=4, strides=2, 
                            kernel_initializer = tf.keras.initializers.GlorotNormal(seed=None), name = 'layer10')(layer9) 
    layer10 = Cropping2D(((1,1),(1,1)))(layer10)
    
    layer10 = InstanceNormalization()(layer10, training=1)
    layer10 = Concatenate(axis=-1)([layer10, layer2])
    layer10 = Activation('relu')(layer10)
    layer10 = concatenate([layer10, xi_yi_sz32]) # ==========
    layer11 = Conv2DTranspose(64, kernel_size=4, strides=2, 
                            kernel_initializer = tf.keras.initializers.GlorotNormal(seed=None), name = 'layer11')(layer10) 
    layer11 = Cropping2D(((1,1),(1,1)))(layer11)
    
    layer11 = InstanceNormalization()(layer11, training=1)
    layer11 = Activation('relu')(layer11)
        
    layer12 = concatenate([layer11, xi_yi_sz64]) # ==========
    layer12 = Activation('relu')(layer12)
    layer12 = Conv2DTranspose(32, kernel_size=4, strides=2, 
                            kernel_initializer = tf.keras.initializers.GlorotNormal(seed=None), name = 'layer12')(layer12) 
    layer12 = Cropping2D(((1,1),(1,1)))(layer12)
    
    layer12 = InstanceNormalization()(layer12, training=1)
    
    layer12 = Conv2D(3, kernel_size=4, strides=1, 
                   padding="same", name = 'out128') (layer12)
    

    alpha = Activation("sigmoid", name='alpha_sigmoid')(layer12)
    
    return Model(inputs=inputs, outputs=alpha) 

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator = build_generator()
generator.summary()

In [ ]:
def build_discriminator():
    input_shape =(256, 256, 3)
    conv_base = tf.keras.applications.vgg16.VGG16(weights=None, include_top=False, input_shape=input_shape)
    model_output = conv_base.output
    x = Flatten()(model_output)
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(conv_base.input, outputs)

In [ ]:
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator = build_discriminator()
discriminator.summary()

In [ ]:
# Compile the models
generator.compile(loss='mae', optimizer=generator_optimizer)
discriminator.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer)

In [ ]:
def generator_loss(disc_output_fake, gen_output, target_image):
    l1_loss = tf.reduce_mean(tf.abs(target_image - gen_output))
    adversarial_loss = tf.reduce_mean(tf.square(disc_output_fake - 1))
    total_loss = l1_loss + (0.01 * adversarial_loss)
    return total_loss

def discriminator_loss(real_output, generated_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    generated_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(generated_output), generated_output)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss

In [ ]:
@tf.function
def train_step(dataset):
    person_image, target_image = dataset
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        person_image = tf.cast(person_image, tf.float32)
        target_image = tf.cast(target_image, tf.float32)

        gen_inputs = person_image
        gen_output = generator(gen_inputs, training=True)
        # Concatenate input and target images for discriminator
        disc_input_real = person_image
        disc_input_fake = gen_output
        disc_output_real = discriminator(disc_input_real, training=True)
        disc_output_fake = discriminator(disc_input_fake, training=True)
        # Calculate losses
        gen_loss = generator_loss(disc_output_fake, gen_output, target_image)
        disc_loss = discriminator_loss(disc_output_real, disc_output_fake)

    # Calculate gradients and update weights
    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))
    return {'gen_loss': gen_loss, 'disc_loss': disc_loss}

In [ ]:
num_epochs=1000
gen_loss_avg = tf.keras.metrics.Mean('gen_loss', dtype=tf.float32)
disc_loss_avg = tf.keras.metrics.Mean('disc_loss', dtype=tf.float32)

for epoch in range(num_epochs):
    print(f"Running Epoch : {epoch+1}")
    for step, image_batch in enumerate(dataset):
        results = train_step(image_batch)
        gen_loss = results['gen_loss']
        disc_loss = results['disc_loss']
        # update the running averages for gen_loss and disc_loss
        gen_loss_avg.update_state(gen_loss)
        disc_loss_avg.update_state(disc_loss)
        if (step + 1) % 10 == 0:
            print('Epoch {}, Step {}, Gen Loss: {:.4f}, Disc Loss: {:.4f}'.format(epoch + 1, step + 1, gen_loss_avg.result(), disc_loss_avg.result()))
    # reset the running averages at the end of each epoch
    gen_loss_avg.reset_states()
    disc_loss_avg.reset_states()

In [ ]:
generator.save("generator.h5")

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load the person and shirt images
person_image = Image.open('1.jpg')

def predict(person_image, generator):
    # Resize person and cloth images to match generator input size
    person_image = tf.image.resize(person_image, [256, 256])

    # Normalize person and cloth images to [0, 1] range
    person_image = person_image / 255.0
    person_image = np.expand_dims(person_image, axis=0)

    output_image = generator(person_image, training=False)
    # Denormalize output image and clip values to [0, 255] range
    output_image = (output_image + 1) * 127.5
    output_image = tf.clip_by_value(output_image, 0, 255)
    # Convert output image to numpy array and return
    output_image = output_image.numpy().astype(np.uint8)
    return output_image


output_image = predict(person_image, generator)
output_image = output_image.squeeze()
plt.imshow(output_image)
plt.axis('off')
plt.show()